In [1]:
# Creamos el DataFrame
import ast
import pandas as pd

rows = []
with open("steam_games.json") as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))

df = pd.DataFrame(rows)

In [3]:
# Seleccionar las columnas deseadas
columnas_deseadas = ["release_date", "genres", "metascore", "price"]

# Crear el nuevo DataFrame solo con las columnas deseadas
df_reduced = df[columnas_deseadas].copy()


In [50]:
df_reduced.tail()

,metascore,price,year,Action,Adventure,Casual,Early Access,Free to Play,Indie,Massively Multiplayer,RPG,Racing,Simulation,Sports,Strategy,Video Production
32109,71.0,4.99,2001.0,1,0,0,0,0,0,0,0,0,0,0,0,0
32112,88.0,14.99,2002.0,1,1,0,0,0,0,0,0,0,0,0,0,0
32114,79.0,4.99,2003.0,1,0,0,0,0,0,0,0,0,0,0,0,0
32116,93.0,14.99,2004.0,1,0,0,0,0,0,0,0,0,0,0,0,0
32117,65.0,9.99,2004.0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
# Convertir la columna "price" a numérica y reemplazar los valores no numéricos con 0
df_reduced['price'] = pd.to_numeric(df['price'], errors='coerce').fillna(0)

In [12]:
# Eliminar filas con valores nulos en la columna "price"
df_reduced.dropna(subset=['price'], inplace=True)

In [14]:
# Eliminar filas con valores nulos en la columna "metascore"
df_reduced.dropna(subset=['metascore'], inplace=True)

In [32]:
import pandas as pd

# Reemplazar los valores "NA" por NaN en la columna "metascore"
df_reduced["metascore"] = df_reduced["metascore"].replace("NA", pd.NA)

# Convertir la columna "metascore" a tipo numérico
df_reduced["metascore"] = pd.to_numeric(df_reduced["metascore"], errors='coerce')

# Eliminar filas con valores NaN en la columna "metascore" del DataFrame "df_reduced"
df_reduced.dropna(subset=["metascore"], inplace=True)


In [38]:
df_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2607 entries, 28 to 32117
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   metascore              2607 non-null   float64
 1   price                  2607 non-null   float64
 2   year                   2553 non-null   float64
 3   Action                 2607 non-null   int64  
 4   Adventure              2607 non-null   int64  
 5   Casual                 2607 non-null   int64  
 6   Early Access           2607 non-null   int64  
 7   Free to Play           2607 non-null   int64  
 8   Indie                  2607 non-null   int64  
 9   Massively Multiplayer  2607 non-null   int64  
 10  RPG                    2607 non-null   int64  
 11  Racing                 2607 non-null   int64  
 12  Simulation             2607 non-null   int64  
 13  Sports                 2607 non-null   int64  
 14  Strategy               2607 non-null   int64  
 15  Vi

In [35]:
# Transformación del año
df_reduced["year"] = pd.to_datetime(df_reduced["release_date"]).dt.year

# Transformación del género utilizando codificación one-hot
generos_dummies = df_reduced["genres"].str.join(",").str.get_dummies(sep=",")
df_reduced = pd.concat([df_reduced, generos_dummies], axis=1)

# Eliminar las columnas originales de "release_date" y "genres"
df_reduced.drop(["release_date", "genres"], axis=1, inplace=True)


In [45]:
# Eliminar filas con valores NaN en la columna "year" del DataFrame "df_reduced"
df_reduced.dropna(subset=["year"], inplace=True)


In [46]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Supongamos que tienes tus datos en un DataFrame llamado "df"
# Eliminar la columna "price" para utilizarla como variable objetivo para la predicción
X = df_reduced.drop(columns=['price'])
y = df_reduced['price']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo de regresión lineal múltiple
model = LinearRegression()

# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Evaluar el rendimiento del modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [47]:
from sklearn.metrics import mean_squared_error
import numpy as np

# Calcular el MSE
mse = mean_squared_error(y_test, y_pred)

# Calcular el RMSE
rmse = np.sqrt(mse)

# Mostrar el RMSE
print("Root Mean Squared Error:", rmse)

Root Mean Squared Error: 8.36414991271523


In [48]:
import pickle

# Guardar el modelo en un archivo pickle
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [51]:
# Calcular las diferencias (residuos)
diferencias = y_test - y_pred

# Crear un DataFrame para visualizar las diferencias junto con las predicciones y los valores reales
resultados = pd.DataFrame({'Precio Real': y_test, 'Predicción': y_pred, 'Diferencia': diferencias})

# Mostrar los ejemplos de 20 a 30
print(resultados.iloc[10:20])

       Precio Real  Predicción  Diferencia
2845         14.99   13.166942    1.823058
28690        14.99   15.319067   -0.329067
18950        44.99   19.424351   25.565649
31840         9.99   11.831373   -1.841373
838           9.99   19.055511   -9.065511
28385         9.99   12.933436   -2.943436
31381         9.99   16.409726   -6.419726
16046        19.99   22.064652   -2.074652
24354         9.99    9.582067    0.407933
142           9.99   13.527662   -3.537662
